# 1&2

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plot
import plotly.graph_objects as go
import statsmodels.api as sm

In [2]:
data = yf.download("^GSPC ^IXIC GLD ^VIX", start="2007-01-01", end="2020-03-31")["Close"]
data.head()

[*********************100%***********************]  4 of 4 completed


Ticker,GLD,^GSPC,^IXIC,^VIX
Date,,,,
2007-01-03 00:00:00+00:00,62.279999,1416.599976,2423.159912,12.04
2007-01-04 00:00:00+00:00,61.650002,1418.339966,2453.429932,11.51
2007-01-05 00:00:00+00:00,60.169998,1409.709961,2434.250000,12.14
2007-01-08 00:00:00+00:00,60.480000,1412.839966,2438.199951,12.00
2007-01-09 00:00:00+00:00,60.849998,1412.109985,2443.830078,11.91


In [3]:
GLD = data[['GLD']].rename(columns={'GLD': 'Close'})
GSPC = data[['^GSPC']].rename(columns={'^GSPC': 'Close'})
IXIC = data[['^IXIC']].rename(columns={'^IXIC': 'Close'})
bear_market_start = "2007-01-01"
bear_market_end = "2009-03-31"
bull_market_start = "2009-03-31"
bull_market_end = "2020-01-31"
covid_crisis_start = "2020-01-31"
covid_crisis_end = "2020-03-31"
stocks = {'GSPC': GSPC, 
          'GLD': GLD, 
          'IXIC': IXIC}
stocks_bear_market = {}
stocks_bull_market = {}
stocks_covid_market = {}

In [4]:
def analyze_period(period_name, stock, start_date, end_date):
    df1 = stock[start_date: end_date]
    df1_returns = df1['Return']
    df1_cum_returns = df1['Cum Return']
    return df1

In [5]:
for stock in stocks.keys():
    stocks[stock]['Return'] = stocks[stock]['Close'].pct_change()
    stocks[stock]['Cum Return'] = (1 + stocks[stock]['Return']).cumprod()
    stocks_bear_market[stock] = analyze_period('bear', stocks[stock], bear_market_start, bear_market_end)
    stocks_bull_market[stock] = analyze_period('bull', stocks[stock], bull_market_start, bull_market_end)
    stocks_covid_market[stock] = analyze_period('covid', stocks[stock], covid_crisis_start, covid_crisis_end)

# 3

In [6]:
stock_cum_return = pd.DataFrame()
for stock in stocks.keys():
    stock_cum_return[stock] = stocks[stock]['Cum Return']
stock_cum_return

,GSPC,GLD,IXIC
Date,,,
2007-01-03 00:00:00+00:00,NaN,NaN,NaN
2007-01-04 00:00:00+00:00,1.001228,0.989884,1.012492
2007-01-05 00:00:00+00:00,0.995136,0.966121,1.004577
2007-01-08 00:00:00+00:00,0.997346,0.971098,1.006207
2007-01-09 00:00:00+00:00,0.996830,0.977039,1.008530
...,...,...,...
2020-03-24 00:00:00+00:00,1.727608,2.463070,3.061234
2020-03-25 00:00:00+00:00,1.747536,2.429351,3.047384
2020-03-26 00:00:00+00:00,1.856607,2.460501,3.217922


In [7]:
def plot_compare(dict1):
    for key in dict1.keys():
        dict1[key]['Cum Return'].plot(figsize = (15,6))
    plot.legend(dict1.keys())

In [8]:
def plot_compare2(fig, dict1, period_name):
    for key in dict1.keys():
        fig.add_trace(go.Scatter(x=dict1[key].index, y=dict1[key]['Cum Return'], mode='lines', name=key))
    fig.update_layout(
        title={'text':'Cumulative Performance of the ' + period_name},
        title_x=0.5, 
        xaxis=dict(
            title='Date',
            ),
        yaxis=dict(
            title='Return',
            ticksuffix="%"
            ),
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)'
    )
    return fig

# 4

In [9]:
fig1 = plot_compare2(go.Figure(), stocks, 'Entire Period')
fig1.show()

In [10]:
fig2 = plot_compare2(go.Figure(), stocks_bear_market, 'Bear Market')
fig2.show()

In [11]:
fig3 = plot_compare2(go.Figure(), stocks_bull_market, 'Bull Market')
fig3.show()

In [12]:
fig4 = plot_compare2(go.Figure(), stocks_covid_market, 'Covid Period')
fig4.show()

In [13]:
stock_return = pd.DataFrame()
stock_return['GSPC'] = data['^GSPC'].pct_change()
stock_return['IXIC'] = data['^IXIC'].pct_change()
stock_return['GLD'] = data['GLD'].pct_change()
stock_return['VIX'] = data['^VIX'].pct_change()
stock_return

,GSPC,IXIC,GLD,VIX
Date,,,,
2007-01-03 00:00:00+00:00,NaN,NaN,NaN,NaN
2007-01-04 00:00:00+00:00,0.001228,0.012492,-0.010116,-0.044020
2007-01-05 00:00:00+00:00,-0.006085,-0.007818,-0.024007,0.054735
2007-01-08 00:00:00+00:00,0.002220,0.001623,0.005152,-0.011532
2007-01-09 00:00:00+00:00,-0.000517,0.002309,0.006118,-0.007500
...,...,...,...,...
2020-03-24 00:00:00+00:00,0.093828,0.081215,0.048530,0.001299
2020-03-25 00:00:00+00:00,0.011535,-0.004524,-0.013690,0.036971
2020-03-26 00:00:00+00:00,0.062414,0.055962,0.012822,-0.046130


In [14]:
#5
def df_subset(df, start, point1, point2, end):
    df1 = df[start : point1]
    df2 = df[point1 : point2]
    df3 = df[point2 : end]
    return pd.DataFrame(df1), pd.DataFrame(df2), pd.DataFrame(df3)

In [15]:
GSPC_bear, GSPC_bull, GSPC_covid = df_subset(stock_return['GSPC'], "2007-01-01", "2009-03-31", "2020-01-31", "2020-03-31")
GLD_bear, GLD_bull, GLD_covid = df_subset(stock_return['GLD'], "2007-01-01", "2009-03-31", "2020-01-31", "2020-03-31")
IXIC_bear, IXIC_bull, IXIC_covid = df_subset(stock_return['IXIC'], "2007-01-01", "2009-03-31", "2020-01-31", "2020-03-31")
VIX_bear, VIX_bull, VIX_covid = df_subset(stock_return['VIX'], "2007-01-01", "2009-03-31", "2020-01-31", "2020-03-31")

# 5 
## Correlation

In [16]:
bear_correlation = pd.concat([GSPC_bear, GLD_bear, IXIC_bear, VIX_bear], axis = 1).corr()
print(bear_correlation)

          GSPC       GLD      IXIC       VIX
GSPC  1.000000  0.026650  0.966570 -0.750564
GLD   0.026650  1.000000  0.001684 -0.069637
IXIC  0.966570  0.001684  1.000000 -0.736483
VIX  -0.750564 -0.069637 -0.736483  1.000000


In [17]:
bull_correlation = pd.concat([GSPC_bull, GLD_bull, IXIC_bull, VIX_bull], axis = 1).corr()
print(bull_correlation)

          GSPC       GLD      IXIC       VIX
GSPC  1.000000  0.010457  0.952222 -0.775549
GLD   0.010457  1.000000 -0.005408  0.010601
IXIC  0.952222 -0.005408  1.000000 -0.752368
VIX  -0.775549  0.010601 -0.752368  1.000000


In [18]:
covid_correlation = pd.concat([GSPC_covid, GLD_covid, IXIC_covid, VIX_covid], axis = 1).corr()
print(covid_correlation)

          GSPC       GLD      IXIC       VIX
GSPC  1.000000  0.176480  0.987217 -0.751337
GLD   0.176480  1.000000  0.174675 -0.040569
IXIC  0.987217  0.174675  1.000000 -0.795287
VIX  -0.751337 -0.040569 -0.795287  1.000000


## Beta

In [19]:
def beta_cal(Y_list, X):
    beta_list = []
    for Y in Y_list:
        model = sm.OLS(Y,X,missing="drop").fit()
        beta_list.append(model.params['GSPC'])
    return beta_list

beta_bear = beta_cal([GLD_bear, VIX_bear], GSPC_bear)
beta_bull = beta_cal([GLD_bull, VIX_bull], GSPC_bull)
beta_covid = beta_cal([GLD_covid, VIX_covid], GSPC_covid)
beta_list = [beta_bear, beta_bear, beta_bear]
print(beta_list)

[[0.02018356097772447, -3.0374940228167615], [0.02018356097772447, -3.0374940228167615], [0.02018356097772447, -3.0374940228167615]]
